Import Libraries

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.transforms import v2
from torch.utils.data import DataLoader

import numpy as np

import os
import sys

Device Setup:
If you have an Nvidia GPU, you need to install CUDA
Otherwise, CPU will be used

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Hardware to be used:", device)

if torch.cuda.is_available():
    print(torch.cuda.get_device_name())

Enter the parent path (e.g., "C:\\Users\\{USERNAME}\\alzheimer-project-ai4all")

Enter path to dataset (*.npz)

Enter path to test dataset (e.g., "assets\Kaggle\alzheimer_mri_preprocessed_dataset\Test_Data")

In [ ]:
parent_path = r"ENTER PARENT PATH"

dataset_dir = fr"{parent_path}\assets\Kaggle\alzheimer_mri_preprocessed_dataset"

train_dir = fr"Train_Data\augmented_adasyn_train_data.npz"
train_path = os.path.join(dataset_dir, train_dir)

val_dir = fr"Validation_Data\val_data.npz"
val_path = os.path.join(dataset_dir, val_dir)

test_dir = fr"Test_Data\test_data.npz"
test_path = os.path.join(dataset_dir, test_dir)

sys.path.append(parent_path)

In [ ]:
classes = ["Non_Demented", "Very_Mild_Demented", "Mild_Demented", "Moderate_Demented"]

Defining the hyper-parameters

In [ ]:
epoches = 25 # implement early stopping
learning_rate = 0.001
batch_size = 8

Loading the datasets

In [ ]:
from alzheimersdetection.AlzheimerDataset import AlzheimerDataset

transform = v2.Compose([v2.ToImage(), v2.ToDtype])

train_dataset = AlzheimerDataset(train_path, transform=transform)
val_dataset = AlzheimerDataset(val_path, transform=transform)
test_dataset = AlzheimerDataset(test_path, transform=transform)

In [ ]:
print("Training Dataset Size:")
len(train_dataset)

print("\nValidation Dataset Size:")
len(val_dataset)

print("\nTesting Dataset Size:")
len(test_dataset)

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

Create our model using PyTorch's Convolutional Neural Network

In [ ]:
from alzheimersdetection.AlzheimerModel import AlzheimerCNN
size = len(train_loader)
model = AlzheimerCNN(size).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

n_total_steps = len(train_loader)

Training Model

In [ ]:
for epoch in range(epoches):
    model.train()
    running_loss = 0.0
    
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()

        # Forward
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward() # Backpropogation

        optimizer.step() # Update the weights!!!

        running_loss+=loss.item()

    epoch_loss = running_loss/len(train_loader)
    print(f"Epoch {epoch + 1}/{epoches} - Training Loss: {epoch_loss:.4f}")


------
<p style="text-align: center;"> Made with ❤️ </p>
<p style="text-align: center;"> Darwin Xue </p>